# Finetuning ResNet50


In [1]:
from datasets import load_dataset

cifar10dataset = load_dataset("cifar10", split="train[:5000]")

In [2]:
cifar10dataset = cifar10dataset.train_test_split(test_size=0.2)

In [3]:
labels = cifar10dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [4]:
from transformers import AutoImageProcessor

checkpoint = "microsoft/resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
2023-08-10 15:17:45.457060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 15:17:46.349205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [6]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["img"]]
    del examples["img"]
    return examples

In [7]:
cifar10dataset = cifar10dataset.with_transform(transforms)

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    ignore_mismatched_sizes=True,
    id2label=id2label,
    label2id=label2id,
)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([10, 2048]) in the model instantiated
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir="cifar10_resnet50",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8, # memory error with 16
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=cifar10dataset["train"],
    eval_dataset=cifar10dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/mahnaz/mlprojects/bloom-classifier/venv_bloom-classifier/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 2.3054, 'learning_rate': 1.3157894736842106e-05, 'epoch': 0.08}
{'loss': 2.3145, 'learning_rate': 2.6315789473684212e-05, 'epoch': 0.16}
{'loss': 2.294, 'learning_rate': 3.9473684210526316e-05, 'epoch': 0.24}
{'loss': 2.301, 'learning_rate': 4.9703264094955494e-05, 'epoch': 0.32}
{'loss': 2.2901, 'learning_rate': 4.821958456973294e-05, 'epoch': 0.4}
{'loss': 2.2883, 'learning_rate': 4.673590504451038e-05, 'epoch': 0.48}
{'loss': 2.2846, 'learning_rate': 4.525222551928784e-05, 'epoch': 0.56}
{'loss': 2.2747, 'learning_rate': 4.3768545994065286e-05, 'epoch': 0.64}
{'loss': 2.2777, 'learning_rate': 4.228486646884273e-05, 'epoch': 0.72}
{'loss': 2.2764, 'learning_rate': 4.080118694362018e-05, 'epoch': 0.8}
{'loss': 2.2716, 'learning_rate': 3.9317507418397627e-05, 'epoch': 0.88}
{'loss': 2.2698, 'learning_rate': 3.783382789317508e-05, 'epoch': 0.96}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.256812810897827, 'eval_accuracy': 0.229, 'eval_runtime': 5.6183, 'eval_samples_per_second': 177.989, 'eval_steps_per_second': 22.249, 'epoch': 1.0}
{'loss': 2.2609, 'learning_rate': 3.635014836795252e-05, 'epoch': 1.04}
{'loss': 2.2675, 'learning_rate': 3.4866468842729974e-05, 'epoch': 1.12}
{'loss': 2.2551, 'learning_rate': 3.338278931750742e-05, 'epoch': 1.2}
{'loss': 2.2541, 'learning_rate': 3.189910979228487e-05, 'epoch': 1.28}
{'loss': 2.2441, 'learning_rate': 3.0415430267062318e-05, 'epoch': 1.36}
{'loss': 2.2456, 'learning_rate': 2.8931750741839762e-05, 'epoch': 1.44}
{'loss': 2.2361, 'learning_rate': 2.744807121661721e-05, 'epoch': 1.52}
{'loss': 2.2345, 'learning_rate': 2.5964391691394662e-05, 'epoch': 1.6}
{'loss': 2.2342, 'learning_rate': 2.4480712166172106e-05, 'epoch': 1.68}
{'loss': 2.2372, 'learning_rate': 2.2997032640949558e-05, 'epoch': 1.76}
{'loss': 2.2298, 'learning_rate': 2.1513353115727002e-05, 'epoch': 1.84}
{'loss': 2.2166, 'learning_rate': 2.002

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.2155098915100098, 'eval_accuracy': 0.296, 'eval_runtime': 5.8047, 'eval_samples_per_second': 172.276, 'eval_steps_per_second': 21.534, 'epoch': 2.0}
{'loss': 2.2164, 'learning_rate': 1.706231454005935e-05, 'epoch': 2.08}
{'loss': 2.2, 'learning_rate': 1.5578635014836794e-05, 'epoch': 2.16}
{'loss': 2.202, 'learning_rate': 1.4094955489614246e-05, 'epoch': 2.24}
{'loss': 2.2195, 'learning_rate': 1.2611275964391692e-05, 'epoch': 2.32}
{'loss': 2.2047, 'learning_rate': 1.112759643916914e-05, 'epoch': 2.4}
{'loss': 2.2022, 'learning_rate': 9.643916913946588e-06, 'epoch': 2.48}
{'loss': 2.2018, 'learning_rate': 8.160237388724036e-06, 'epoch': 2.56}
{'loss': 2.193, 'learning_rate': 6.676557863501484e-06, 'epoch': 2.64}
{'loss': 2.1873, 'learning_rate': 5.192878338278932e-06, 'epoch': 2.72}
{'loss': 2.192, 'learning_rate': 3.7091988130563796e-06, 'epoch': 2.8}
{'loss': 2.186, 'learning_rate': 2.225519287833828e-06, 'epoch': 2.88}
{'loss': 2.1992, 'learning_rate': 7.418397626112

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.192864179611206, 'eval_accuracy': 0.293, 'eval_runtime': 5.7669, 'eval_samples_per_second': 173.402, 'eval_steps_per_second': 21.675, 'epoch': 3.0}
{'train_runtime': 219.6975, 'train_samples_per_second': 54.621, 'train_steps_per_second': 1.707, 'train_loss': 2.24185915629069, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=2.24185915629069, metrics={'train_runtime': 219.6975, 'train_samples_per_second': 54.621, 'train_steps_per_second': 1.707, 'train_loss': 2.24185915629069, 'epoch': 3.0})

Model is learning as the loss is decreasing, however very slowly. why is that, and how we can make the learning faster?